In [1]:
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.experimental import enable_hist_gradient_boosting

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import PoissonRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.linear_model import GammaRegressor
from sklearn.linear_model import LogisticRegression

from joblib import dump, load
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import RobustScaler
import pandas as pd
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
plt.style.use('seaborn')
%matplotlib inline

pd.set_option('display.max_columns', None)

In [2]:
df=pd.read_parquet('../data/processed/df.parquet')
dfd=pd.read_parquet('../data/processed/dfd.parquet')


In [3]:
dfd.columns

Index(['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'df_original', 'id',
       'cut_int', 'color_int', 'clarity_int', 'cut_color_clarity_score',
       'raw_density', 'marketing_carat_int', 'carat_sqr', 'cut_Fair',
       'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_D',
       'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J',
       'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2', 'marketing_carat_bad',
       'marketing_carat_good', 'marketing_carat_regular'],
      dtype='object')

In [4]:
#option 1

# dfd=dfd.drop(columns=['marketing_carat_int','cut_int','color_int', 'clarity_int'])

# #option 2
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score'])

# #option 3 espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','x', 'y', 'z'])

# #option 3 bis espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','raw_density'])

# #option 3 bis2 espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'x', 'y', 'z'])

# #option 4
# dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','x', 'y', 'z','raw_density'])

# #option 5 SIN NADA
# dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','raw_density','marketing_carat_bad',
#        'marketing_carat_good', 'marketing_carat_regular'])

#option 5 SIN NADA
dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
                     'marketing_carat_bad',
       'marketing_carat_good', 'marketing_carat_regular','carat_sqr','cut_color_clarity_score'])

In [5]:
#con carat y 5
# RandomForestRegressor       536.7894862874348
# ExtraTreesRegressor         524.3073747284154
# GradientBoostingRegressor   522.2696670624822
# CatBoostRegressor           515.8518906580055
# LGBMRegressor               520.4246279374379
# XGBRegressor                517.9044507009544
# StackingRegressor           504.8044508186878

#sin carat y 5
# RandomForestRegressor       536.5326496454497
# ExtraTreesRegressor         522.8754833840452
# GradientBoostingRegressor   521.8840694917378
# CatBoostRegressor           514.4646192634776
# LGBMRegressor               520.4418909188064
# XGBRegressor                517.6058680138955
# StackingRegressor           504.43703022069695


df=df.drop(columns=['cut','color', 'clarity','marketing_carat','marketing_carat_int','cut_color_clarity_score','carat'])


#RandomForestRegressor        536.8794322366753
# ExtraTreesRegressor         522.5880641009147
# GradientBoostingRegressor   521.8113311805382
# CatBoostRegressor           515.7005012415111
# LGBMRegressor               520.4246279374379
# XGBRegressor
# StackingRegressor           505.6372856024203

# df=df.drop(columns=['cut','color', 'clarity','marketing_carat','marketing_carat_int','cut_color_clarity_score','carat_sqr'])



In [6]:
df.head()


,depth,table,price,x,y,z,df_original,id,cut_int,color_int,clarity_int,raw_density,carat_sqr
0,62.4,58.0,4268.0,6.83,6.79,4.25,1,NaN,4,1,4,3.683471,2.593742
1,63.0,57.0,505.0,4.35,4.38,2.75,1,NaN,3,3,4,3.664419,0.003355
2,65.5,55.0,2686.0,5.62,5.53,3.65,1,NaN,1,4,5,3.755392,0.180423
3,63.8,56.0,738.0,4.68,4.72,3.00,1,NaN,2,7,3,3.712154,0.011586
4,60.5,59.0,4882.0,6.55,6.51,3.95,1,NaN,5,4,3,3.633558,1.104081


In [7]:
dfd.head()

,carat,depth,table,price,x,y,z,df_original,id,raw_density,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.21,62.4,58.0,4268.0,6.83,6.79,4.25,1,NaN,3.683471,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,0.32,63.0,57.0,505.0,4.35,4.38,2.75,1,NaN,3.664419,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,0.71,65.5,55.0,2686.0,5.62,5.53,3.65,1,NaN,3.755392,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,0.41,63.8,56.0,738.0,4.68,4.72,3.00,1,NaN,3.712154,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,1.02,60.5,59.0,4882.0,6.55,6.51,3.95,1,NaN,3.633558,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


## Split

In [8]:


df_final=df

data=df_final.loc[df['df_original']==1]

############## FILAS


# data=data.drop_duplicates(subset=data.columns, keep='first')

data['depth_calc'] = data['z'] / ((data['x'] + data['y'])/2)*100
data['rest']=data['depth_calc'] - data['depth'] 
filter_rest=(data['rest']<-1) | (data['rest']>1)
data=data.loc[~filter_rest]
data=data.drop(columns=['depth_calc','rest'],axis=1)

filter_xyz=(data['x']<0.2) | (data['y']<0.2) | (data['z']<0.2)
data=data.loc[~filter_xyz ]



df_predict=df_final.loc[df['df_original']==0]

X=data.drop(columns=['price','df_original','id'])
Y=data['price']

X_predict=df_predict.drop(columns=['price','df_original','id'])


rs = RobustScaler() 
X = pd.DataFrame(rs.fit_transform(X), columns= X.columns)
X_predict = pd.DataFrame(rs.fit_transform(X_predict), columns= X_predict.columns)

df_predict.to_csv('../data/processed/df_predict.csv',index=False)
X.to_csv('../data/processed/X.csv',index=False)
Y.to_csv('../data/processed/Y.csv',index=False)
X_predict.to_csv('../data/processed/X_predict.csv',index=False)



print(data.shape)
print(df_predict.shape)
print(X.shape)
print(X_predict.shape)



/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


(40380, 13)
(13485, 13)
(40380, 10)
(13485, 10)


## Model Generator

In [14]:
# arboles  de decision max_depth probar min sample split
# RandomForestRegressor(max_depth=40, min_samples_split=4, n_estimators=500, random_state=32)
# ('RandomForestRegressor', RandomForestRegressor(max_depth=32, min_samples_split=4, n_estimators=250, random_state=32)),
# max_depth=-1


estimators=[('XGBRegressor' , XGBRegressor(base_score=0.8 ,booster = 'gbtree' ,learning_rate=0.05,max_depth= 6 ,n_estimators=500 ,random_state=32 )),
    ('LGBM' , LGBMRegressor(learning_rate =  0.05, n_estimators =  435, max_depth=12, random_state = 32)),
             ('ExtraTreesRegressor',ExtraTreesRegressor(max_depth=16, min_samples_split=6, n_estimators=400,n_jobs = -1, random_state=32)),
           ('GradientBoostingRegressor',GradientBoostingRegressor(max_depth=8, min_samples_split=12, n_estimators=80,random_state=32)),
            ('CatBoostRegressor',CatBoostRegressor(depth = 8, iterations = 1200,silent=True,random_state = 32)),
            
        ]

# ('XGBRegressor' , XGBRegressor(base_score=0.8 ,booster = 'gbtree' ,learning_rate=0.05,max_depth= 8 ,n_estimators=300 ,random_state=32 ))

machine = [
           ExtraTreesRegressor(max_depth=16, min_samples_split=6, n_estimators=400,n_jobs = -1, random_state=32),
           GradientBoostingRegressor(max_depth=8, min_samples_split=12, n_estimators=80,random_state=32),
           CatBoostRegressor(depth = 8, iterations = 1200,silent=True,random_state = 32),
           LGBMRegressor( learning_rate =  0.05, n_estimators =  435, max_depth=12, random_state = 32),
           XGBRegressor(base_score=0.8 ,booster = 'gbtree' ,learning_rate=0.05,max_depth= 6 ,n_estimators=500 ,random_state=32 ),
          StackingRegressor(estimators=estimators,final_estimator= LinearRegression(n_jobs = -1),
                               cv= 5 , verbose = 10 , passthrough = True),
          ]  


# machine = [  StackingRegressor(estimators=estimators,final_estimator= LinearRegression(n_jobs = -1) ,cv= 5 , 
#                                verbose = 2 , passthrough = True)]  

# LinearRegression(n_jobs = -1)
# ARDRegression(n_iter=1000)

machine = [ GradientBoostingRegressor(max_depth=6, min_samples_leaf=6, min_samples_split=16, n_estimators=200,random_state=32) ]








for x in machine:
    print(f'Model in action: {x}\n')
    
#     X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.85, random_state=32)
    
######################           ENTRENAR AL FINAL CON TODO


    
#     features = RFECV(x)
#     features= features.fit(X_train, Y_train)
#     features_columns=[X.columns[pos] for pos,item in enumerate(features.support_.tolist()) if item==False]
#     X_train=X_train.drop(columns=features_columns)
#     X_test=X_test.drop(columns=features_columns)
#     X_predict=df_predict.drop(columns=['price','df_original','id'])
#     X_predict=X_predict.drop(columns=features_columns)
#     print(f'Delete columns: {len(X.columns)-features.n_features_}  {features_columns} ')
        
        
#     model = x  
#     model.fit(X_train, Y_train)
    
    
#     predictions_train = model.predict(X_train)#.clip(326,18823)
#     predictions_test = model.predict(X_test)#.clip(326,18823)
 

    
#     print(f' R2 Train: {r2_score(y_true=Y_train, y_pred=predictions_train)}')
#     print(f' R2 Test: {r2_score(y_true=Y_test, y_pred=predictions_test)}')
#     print(f' RMSE = {(mean_squared_error(Y_test, predictions_test,squared=False))}\n\n')
    
    
    model = x  

    
    
#     model.fit(X,Y)


    scores = cross_val_score(model, X, Y, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)
    print(f' RMSE_CROSS = {np.mean(-scores)}\n\n')




# RandomForestRegressor       536.5326496454497
# ExtraTreesRegressor         522.0193895803538
# GradientBoostingRegressor   521.8840694917378
# CatBoostRegressor           514.4646192634776 cambie randomestate
# LGBMRegressor               520.4418909188064
# XGBRegressor                517.6058680138955
# StackingRegressor           504.43703022069695 5 con 4 504.63291210793216



Model in action: GradientBoostingRegressor(max_depth=6, min_samples_leaf=6, min_samples_split=16,
                          n_estimators=200, random_state=32)

 RMSE_CROSS = 517.7431819740052




In [ ]:
5



## RandomizedSearchCV

In [9]:


model2= GradientBoostingRegressor(random_state=32)


#  'n_estimators': [16, 32, 64, 128, 256, 500],
#     'min_samples_split': [2, 8, 10, 12],
#     'max_depth' : [2, 4, 8, 10, 16, 32, 50, 75]
# min_samples_split
# min_samples_leaf=1
#leaves

param_grid = {
'max_depth' :[4,6,8,16], 
    'min_samples_split' :[14,16,18], 
    'n_estimators':[120,200,400],
    'min_samples_leaf' : [4,6,8],
}

grid_search = GridSearchCV(model2, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=10, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1,
                                )
# n_iter=32
grid_search.fit(X, Y)

Fitting 5 folds for each of 225 candidates, totalling 1125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   37.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 17.6min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 19.6min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 23.8min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 27.4min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 30.5min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 34.8min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 37

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=32),
             n_jobs=-1,
             param_grid={'max_depth': [6, 8, 16],
                         'min_samples_leaf': [1, 2, 4, 6, 8],
                         'min_samples_split': [10, 12, 16],
                         'n_estimators': [80, 120, 200, 400, 600]},
             scoring='neg_root_mean_squared_error', verbose=10)

In [10]:
grid_search.best_params_

{'max_depth': 6,
 'min_samples_leaf': 6,
 'min_samples_split': 16,
 'n_estimators': 200}

In [11]:
grid_search.best_estimator_
# (learning_rate=0.05, max_depth=12, n_estimators=430,random_state=32)

GradientBoostingRegressor(max_depth=6, min_samples_leaf=6, min_samples_split=16,
                          n_estimators=200, random_state=32)

In [12]:
grid_search.best_score_
# -520.5383378789018

-517.7431819740052

## PROBAR RESULTADO

In [319]:
# dfX_test=pd.DataFrame(X_test)
# dfY_test=pd.DataFrame(Y_test)
# dfpredictions_test=pd.DataFrame(predictions_test)
# dftest=pd.concat([dfX_test.reset_index(drop=True),dfY_test.reset_index(drop=True),dfpredictions_test.reset_index(drop=True)], axis=1) 
# dftest.rename(columns={0: 'price_predict'},inplace=True)
# dftest['Diff']=dftest['price']-dftest['price_predict']

# print(dftest['Diff'].describe())
# dftest.loc[dftest['Diff']>2000].describe()

count      1.000000
mean     859.061216
std             NaN
min      859.061216
25%      859.061216
50%      859.061216
75%      859.061216
max      859.061216
Name: Diff, dtype: float64


,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,cut_color_clarity_score,raw_density,price,price_predict,Diff
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [278]:
# dump(model, '/home/carpiero/ir/Kaggle/.joblib') 

predictions_competition = model.predict(X_predict)#.clip(300,22000)
submission = pd.DataFrame({'id': df_predict['id'], 'price': predictions_competition})
submission['id'] = submission['id'].astype('int64')

submission.to_csv('/home/carpiero/IRONHACK/kaggle/submission.csv', index=False)

PROBA=pd.concat([X_predict,submission], axis=1)



submission


,id,price
0,0,2926.708949
1,1,5513.726681
2,2,9521.893348
3,3,4020.337224
4,4,1629.254509
...,...,...
13480,13480,1661.596852
13481,13481,2307.643773
13482,13482,3046.651081
13483,13483,2063.617314


In [279]:
PROBA.shape

(13485, 12)

In [280]:
PROBA

,depth,table,x,y,z,cut_int,color_int,clarity_int,raw_density,carat_sqr,id,price
0,0.533333,1.000000,0.066667,0.094444,0.125000,-0.5,0.333333,-0.5,1.145788,0.130566,0,2926.708949
1,-0.600000,0.000000,0.616667,0.650000,0.580357,0.5,-1.000000,0.5,0.007319,1.602348,1,5513.726681
2,0.200000,1.333333,0.933333,0.888889,0.928571,0.0,-0.333333,-0.5,1.710806,5.099924,2,9521.893348
3,1.266667,-1.000000,0.216667,0.227778,0.330357,-0.5,0.333333,-0.5,0.453914,0.363554,3,4020.337224
4,0.666667,0.333333,-0.361111,-0.350000,-0.303571,-0.5,0.333333,0.5,-0.138673,-0.155210,4,1629.254509
...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.000000,-0.333333,-0.194444,-0.222222,-0.205357,0.5,0.666667,-0.5,-0.344092,-0.117578,13480,1661.596852
13481,0.200000,-0.666667,0.005556,0.005556,0.026786,0.5,-0.666667,0.0,-0.153810,0.012250,13481,2307.643773
13482,-0.200000,-0.666667,0.027778,-0.005556,0.000000,0.5,0.333333,0.5,-0.548675,0.000000,13482,3046.651081
13483,-2.066667,0.000000,0.083333,0.094444,-0.071429,-0.5,0.333333,-1.0,-1.617572,0.000000,13483,2063.617314


In [281]:
PROBA['price'].describe()

count    13485.000000
mean      3923.473850
std       3950.759154
min        310.220011
25%        929.974643
50%       2425.591199
75%       5261.049745
max      19869.856812
Name: price, dtype: float64

In [350]:
PROBA.loc[(PROBA['price']>19000) | (PROBA['price']<0)]

,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,cut_color_clarity_score,raw_density,id,price
1665,-0.296875,-0.066667,-0.666667,-0.305556,14.488889,1.419643,0.5,0.666667,0.5,0.923077,-38.942974,1665,26550.198901


In [351]:
PROBA.loc[(PROBA['price']>18000) | (PROBA['price']<0)].describe()

,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,cut_color_clarity_score,raw_density,id,price
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,1.515625,0.346667,-0.020000,0.810000,3.765556,6.308929,-0.40000,-0.066667,0.300000,-0.123077,-15.040133,8291.600000,19810.287559
std,1.747068,1.268770,0.804018,1.041614,6.051083,10.588265,0.74162,0.760117,1.151086,1.116843,21.826469,4119.772117,3768.563549
min,-0.296875,-0.400000,-0.766667,-0.322222,-0.316667,1.276786,-1.50000,-1.000000,-1.000000,-1.846154,-38.942974,1665.000000,18045.031457
25%,-0.296875,-0.333333,-0.666667,-0.305556,1.327778,1.419643,-0.50000,-0.666667,-0.500000,-0.615385,-38.942974,7336.000000,18047.028786
50%,2.078125,-0.066667,-0.333333,1.322222,1.538889,1.491071,-0.50000,0.000000,0.500000,0.461538,0.149873,9724.000000,18196.350421
75%,2.484375,-0.066667,0.666667,1.527778,1.788889,2.116071,0.00000,0.666667,0.500000,0.461538,0.907204,10341.000000,18212.828232
max,3.609375,2.600000,1.000000,1.827778,14.488889,25.241071,0.50000,0.666667,2.000000,0.923077,1.628206,12392.000000,26550.198901


In [35]:
p='holaaa'

In [36]:
p[:2]

'ho'